### Data Processing

Xử lý dữ liệu là công đoạn biến đổi dữ liệu thô thành dạng sẵn sàng cho quá trình khai phá. Chúng ta sẽ sử dụng một bộ dữ liệu cụ thể và từng bước xử lý dữ liệu như làm sạch, chuẩn hóa và mã hóa dữ liệu.

#### 1. Tải bộ Titanic từ Kaggle
Chứa thông tin về hành khách trên con tàu Titanic, bao gồm các thuộc tính như tuổi, giới tính, giá vé, hạng vè, và sống sót hay không.

##### Khám phá dữ liệu

In [8]:
import pandas as pd 

# Tải dữ liệu Titanic từ thư mục hoặc URL
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print(df.info())
print(df.describe())

print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

#### 2. Làm sạch dữ liệu (Data Cleaning)
##### 2.1 Xử lý giá trị thiếu
+ Cột Age có nhiều giá trị thiếu
+ Cột Cabin có nhiều giá trị trống, có thể bỏ qua

**Điền giá trị thiếu cho cột** `age` bằng trung bình

In [9]:
df['Age'].fillna(df['Age'].mean(), inplace=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_2016\1698716155.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)


**Loại bỏ cột không cần thiết**

In [11]:
df.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

#### 3. Mã hóa biến phân loại (Categorical Encoding)
##### 3.1 Mã hóa biến giới tính
Chuyển đổi `male` vvaf `female` thành 0 và 1

In [12]:
df['Sex'] = df['Sex'].map({'male':0, 'female':1})

##### 3.2 Mã hóa biến `Embarked` (Nơi lên tàu)
Sử dụng One-hot Encoding 

In [13]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

#### 4. Chuẩn hóa dữ liệu (Normalization)
Chuẩn hóa các cột số (như `Age` và `Fare`) để đưa chúng về cùng thang đo

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

#### 5. Chia dữ liệu thành tập huấn luyện và kiểm tra
Dữ liệu được chia thành tập huấn luyện 70% và tập kiểm tra 30% để huấn luyện và đánh giá mô hình

In [15]:
from sklearn.model_selection import train_test_split

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 6. Huấn luyện mô hình đơn giản (Random Forest)
Áp dụng Random Forest Classifier để dự đoán khả năng sống sót

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [17]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Độ chính xác của mô hình: {accuracy:.2f}")

Độ chính xác của mô hình: 0.83
